# Solution of scheduling problem using State-of-the-art Google ortools libraries

In [18]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 24.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [19]:
!pip install -U pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [20]:
import collections
from ortools.sat.python import cp_model

In [21]:
jobs_data = [[(0, 3), (1, 1), (2, 4), (3, 12)],
 [(0, 8), (1, 0), (2, 5), (3, 15)],
 [(0, 11), (1, 3), (2, 8), (3, 10)],
 [(0, 4), (1, 7), (2, 3), (3, 8)],
 [(0, 5), (1, 5), (2, 1), (3, 10)],
 [(0, 10), (1, 2), (2, 0), (3, 13)],
 [(0, 2), (1, 5), (2, 6), (3, 9)]]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)
# Computes horizon dynamically as the sum of all durations.  in short max possible
horizon = sum(task[1] for job in jobs_data for task in job)

#### `CpModel`: Methods for creating models, including variables and constraints.

#### Here the model to solve the problem is declared in cell below

In [22]:
model = cp_model.CpModel()

#### `collections.namedtuple` create tuple like object with name of object
#### `collections.defaultdict` create a special kind of dictionary, easy to query and update.
#### `model.NewIntVar` to define our variable 
#### `model.NewIntervalVar` to define an varible with interval(size) such that start + size = end






In [23]:
# Named tuple to store information about created variables.
task_type = collections.namedtuple('task_type', 'start end interval')
# Named tuple to manipulate solution information.
assigned_task_type = collections.namedtuple('assigned_task_type',
                                            'start job index duration')

# Creates job intervals and add to the corresponding machine lists.
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        #Creating variables with lb & ub
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        #this NewINtervalVar - An interval variable is a constraint, that is itself used 
        #in other constraints like NoOverlap. Internally, it ensures that start + size == end.
        interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                            'interval' + suffix)
        all_tasks[job_id, task_id] = task_type(start=start_var,
                                               end=end_var,
                                               interval=interval_var)
        machine_to_intervals[machine].append(interval_var)

In [24]:
sorted(machine_to_intervals.items())

[(0,
  [interval_0_0(start = start_0_0, size = 3, end = end_0_0),
   interval_1_0(start = start_1_0, size = 8, end = end_1_0),
   interval_2_0(start = start_2_0, size = 11, end = end_2_0),
   interval_3_0(start = start_3_0, size = 4, end = end_3_0),
   interval_4_0(start = start_4_0, size = 5, end = end_4_0),
   interval_5_0(start = start_5_0, size = 10, end = end_5_0),
   interval_6_0(start = start_6_0, size = 2, end = end_6_0)]),
 (1,
  [interval_0_1(start = start_0_1, size = 1, end = end_0_1),
   interval_1_1(start = start_1_1, size = 0, end = end_1_1),
   interval_2_1(start = start_2_1, size = 3, end = end_2_1),
   interval_3_1(start = start_3_1, size = 7, end = end_3_1),
   interval_4_1(start = start_4_1, size = 5, end = end_4_1),
   interval_5_1(start = start_5_1, size = 2, end = end_5_1),
   interval_6_1(start = start_6_1, size = 5, end = end_6_1)]),
 (2,
  [interval_0_2(start = start_0_2, size = 4, end = end_0_2),
   interval_1_2(start = start_1_2, size = 5, end = end_1_2),
   

In [25]:
all_tasks

{(0,
  0): task_type(start=start_0_0(0..170), end=end_0_0(0..170), interval=interval_0_0(start = start_0_0, size = 3, end = end_0_0)),
 (0,
  1): task_type(start=start_0_1(0..170), end=end_0_1(0..170), interval=interval_0_1(start = start_0_1, size = 1, end = end_0_1)),
 (0,
  2): task_type(start=start_0_2(0..170), end=end_0_2(0..170), interval=interval_0_2(start = start_0_2, size = 4, end = end_0_2)),
 (0,
  3): task_type(start=start_0_3(0..170), end=end_0_3(0..170), interval=interval_0_3(start = start_0_3, size = 12, end = end_0_3)),
 (1,
  0): task_type(start=start_1_0(0..170), end=end_1_0(0..170), interval=interval_1_0(start = start_1_0, size = 8, end = end_1_0)),
 (1,
  1): task_type(start=start_1_1(0..170), end=end_1_1(0..170), interval=interval_1_1(start = start_1_1, size = 0, end = end_1_1)),
 (1,
  2): task_type(start=start_1_2(0..170), end=end_1_2(0..170), interval=interval_1_2(start = start_1_2, size = 5, end = end_1_2)),
 (1,
  3): task_type(start=start_1_3(0..170), end=end_

#### `model.AddNoOverlap` to set no overlap constain
#### `model.AddNoOverlap` to set constomized constain

In [26]:
# Create and add disjunctive constraints.
# AddNoOverlap -- Return type: Constraint
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])   # constain : no overlap 

# Precedences inside a job.
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id +    #  constain : the end time of a task to occur before the start time of the next task in the job.
                            1].start >= all_tasks[job_id, task_id].end)


#### `model.AddMaxEquality` the varible in this will take maximum value from the list of values provided 
#### `model.Minimize` minimizes the given variable (here obj_var) 

In [27]:
# Makespan objective.
obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [ all_tasks[job_id, len(job) - 1].end        #creates a variable obj_var whose value is the maximum 
                                for job_id, job in enumerate(jobs_data)])  #of the end times for all jobs 
                             
model.Minimize(obj_var)

#### `.CpSolver` to create an object which will execute solving and optimizing task of our model.
#### `.Solve` it will solve the defined model, while taking care of all the constrains

In [28]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [29]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:   
    print('Solution:')
    # Create one list of assigned tasks per machine.
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(start=solver.Value(
                    all_tasks[job_id, task_id].start),
                                   job=job_id,
                                   index=task_id,
                                   duration=task[1]))

    # Create per machine output lines.
    output = ''
    for machine in all_machines:
        # Sort by starting time.
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Machine ' + str(machine) + ': '
        sol_line = '           '

        for assigned_task in assigned_jobs[machine]:
            name = 'job_%i_task_%i' % (assigned_task.job,
                                       assigned_task.index)
            # Add spaces to output to align columns.
            sol_line_tasks += '%-15s' % name

            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            # Add spaces to output to align columns.
            sol_line += '%-15s' % sol_tmp

        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    # Finally print the solution found.
    print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
    print(output)
else:
    print('No solution found.')

Solution:
Optimal Schedule Length: 85.0
Machine 0: job_0_task_0   job_6_task_0   job_2_task_0   job_5_task_0   job_1_task_0   job_3_task_0   job_4_task_0   
           [0,3]          [3,5]          [5,16]         [16,26]        [26,34]        [34,38]        [38,43]        
Machine 1: job_0_task_1   job_6_task_1   job_2_task_1   job_5_task_1   job_1_task_1   job_3_task_1   job_4_task_1   
           [3,4]          [5,10]         [16,19]        [26,28]        [34,34]        [38,45]        [45,50]        
Machine 2: job_0_task_2   job_6_task_2   job_2_task_2   job_5_task_2   job_1_task_2   job_3_task_2   job_4_task_2   
           [4,8]          [10,16]        [19,27]        [28,28]        [34,39]        [45,48]        [50,51]        
Machine 3: job_0_task_3   job_6_task_3   job_2_task_3   job_5_task_3   job_4_task_3   job_3_task_3   job_1_task_3   
           [8,20]         [20,29]        [29,39]        [39,52]        [52,62]        [62,70]        [70,85]        




# Google code ends here
--------



### Defining makespan of a sequence of algorithm

In [74]:

def makespan(sequence,processing_times):

    finish_times = [0] * len(processing_times[0])

    # Iterate through the jobs in the given sequence and update the finish times for each machine.
    for job_index in sequence:
        for machine_index in range(len(processing_times[job_index])):
            processing_time = processing_times[job_index][machine_index]
            # print(machine_index)
            if machine_index == 0:
                finish_times[machine_index] += processing_time
            
            else:
                finish_times[machine_index] = max(finish_times[machine_index], finish_times[machine_index-1]) + processing_time

    # The makespan is the maximum finish time across all machines.
    makespan = max(finish_times)

    # print("Makespan:", makespan)
    return makespan

processing_times = [
    [3, 1, 4, 12],
    [8, 0, 5, 15],
    [11, 3, 8, 10],
    [4, 7, 3, 8],
    [5, 5, 1, 10],
    [10, 2, 0, 13],
    [2, 5, 6, 9]
]

sequence = [0, 3, 1, 4, 5, 6, 2]
# 1257436

makespan(sequence,processing_times)

85

# Genetic Heuristic Approch 
Here is a more detailed explanation of the code:

The algorithm takes as input the number of jobs, the number of machines, the processing times for each job on each machine, the population size, the number of generations, the tournament size for selection, the probability of mutation, and the number of local search iterations.

The algorithm initializes the population by creating a list of sequences, where each sequence represents a possible order of jobs to be processed. The sequences are randomly shuffled using the shuffle function from the random module.

The algorithm enters the main loop, which repeats for the specified number of generations.

In each generation, the fitness of each individual in the population is evaluated using the evaluate_fitness function, which computes the makespan of the corresponding sequence using a simulation of the job shop scheduling problem. The fitness scores are sorted in ascending order, and the best sequence is selected as the first element of the sorted population.

The algorithm creates the next generation by selecting pairs of parents using tournament selection with the specified tournament size. Two offspring are created by applying two-point crossover, which randomly selects two points in the parents' sequences and swaps the jobs between the points. Each offspring has a probability of mutation, which swaps two jobs in its sequence. The two offspring are added to the next generation. This process is repeated until the next generation has the same size as the current population.

The old population is replaced with the new generation.

The algorithm applies a local search to the best individual in the population to improve its fitness. The local search iteratively swaps pairs of jobs in the sequence and selects the best swap that reduces the makespan. This process is repeated for the specified number of local search iterations.

The algorithm outputs the best sequence found after the specified number of generations.

In [75]:
import random

def generate_population(pop_size, num_jobs):
    """Generate a population of candidate solutions."""
    population = []
    for _ in range(pop_size):
        sequence = list(range(num_jobs))
        random.shuffle(sequence)
        population.append(sequence)
    return population

def evaluate_fitness(sequence,processing_time,machines = True):
    """Compute the makespan of a given sequence of jobs."""
    finish_times = [0] * len(processing_times[0])

    # Iterate through the jobs in the given sequence and update the finish times for each machine.
    for job_index in sequence:
        for machine_index in range(len(processing_times[job_index])):
            processing_time = processing_times[job_index][machine_index]
            # print(machine_index)
            if machine_index == 0:
                finish_times[machine_index] += processing_time
            
            else:
                finish_times[machine_index] = max(finish_times[machine_index], finish_times[machine_index-1]) + processing_time

    # The makespan is the maximum finish time across all machines.
    makespan = max(finish_times)

    # print("Makespan:", makespan)
    return makespan

def tournament_selection(population, fitness_scores, tournament_size):
    """Select a candidate solution using tournament selection."""
    candidates = random.sample(population, tournament_size)
    return max(candidates, key=lambda x: fitness_scores[population.index(x)])

def two_point_crossover(parent1, parent2):
    """Generate offspring using two-point crossover."""
    num_jobs = len(parent1)
    start = random.randint(0, num_jobs-1)
    end = random.randint(start+1, num_jobs)
    offspring = [-1] * num_jobs
    offspring[start:end] = parent1[start:end]
    for i in range(num_jobs):
        if offspring[i] == -1:
            gene = parent2[i]
            while gene in offspring:
                gene = parent2[parent1.index(gene)]
            offspring[i] = gene
    return offspring

def mutation(sequence, mutation_rate):
    """Mutate a candidate solution by swapping two jobs."""
    if random.random() < mutation_rate:
        num_jobs = len(sequence)
        i = random.randint(0, num_jobs-1)
        j = random.randint(0, num_jobs-1)
        sequence[i], sequence[j] = sequence[j], sequence[i]

def genetic_algorithm(jobs, machines, pop_size=100, num_generations=100, tournament_size=2, mutation_rate=0.1):
    """Solve the job shop scheduling problem using a genetic algorithm."""
    num_jobs = len(jobs)
    population = generate_population(pop_size, num_jobs)
    for generation in range(num_generations):
        fitness_scores = [evaluate_fitness(sequence, jobs, machines) for sequence in population]
        ranked_population = [x for _, x in sorted(zip(fitness_scores, population))]   #ranking the squences
        # print(ranked_population)
        best_sequence = ranked_population[0]
        next_generation = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(ranked_population, fitness_scores, tournament_size)
            parent2 = tournament_selection(ranked_population, fitness_scores, tournament_size)
            offspring1 = two_point_crossover(parent1, parent2)
            offspring2 = two_point_crossover(parent2, parent1)
            mutation(offspring1, mutation_rate)
            mutation(offspring2, mutation_rate)
            next_generation.extend([offspring1, offspring2])
        population = next_generation
    return best_sequence


In [80]:
processing_times = [
    [3, 1, 4, 12],
    [8, 0, 5, 15],
    [11, 3, 8, 10],
    [4, 7, 3, 8],
    [5, 5, 1, 10],
    [10, 2, 0, 13],
    [2, 5, 6, 9]
]
print("Optimal Sequence:",genetic_algorithm(processing_times, machines = True))
print("Makespan:",makespan(genetic_algorithm(processing_times, machines = True),processing_times))

Optimal Sequence: [0, 1, 5, 2, 6, 4, 3]
Makespan: 85


# Jhonsons Heuristic Approch

Johnson's algorithm is a heuristic algorithm used for scheduling jobs in a job-shop environment where each job consists of a set of operations that need to be processed on different machines. The goal is to determine the order in which the jobs should be processed on the machines so as to minimize the total time required to complete all the jobs, also known as the makespan.

The algorithm works by first identifying the machine with the shortest processing time for each job. It then splits the jobs into two groups based on the machines with the shortest processing time: those with the shortest processing time on machine 1 and those with the shortest processing time on machine m.

Next, it sorts the first group in increasing order of their processing time on machine 1 and sorts the second group in decreasing order of their processing time on machine m. This ensures that the jobs with the shortest processing time on machines 1 and m are scheduled first and last, respectively.

The algorithm then iteratively selects the job with the shortest processing time on either machine 1 or m, schedules it on the corresponding machine, and removes it from the list. This process continues until all jobs have been scheduled.

Johnson's algorithm is a heuristic algorithm and does not always guarantee an optimal solution. However, it has been shown to be effective in practice and is often used as a benchmark for evaluating the performance of other scheduling algorithms.

In [83]:
def johnsons_algorithm(n, m, processing_times):
    # Step 1: find minimum and maximum processing times for each job
    min_processing_times = [min(processing_times[i]) for i in range(n)]
    # print(min_processing_times)
    max_processing_times = [max(processing_times[i]) for i in range(n)]
    # print(max_processing_times)

    # Step 2: identify job with minimum maximum processing time
    min_max_processing_time = float('inf')
    for i in range(n):
        if max_processing_times[i] < min_max_processing_time:
            min_max_processing_time = max_processing_times[i]
            min_max_job = i
    # print(min_max_job)
    # Step 3: schedule job first or last depending on which machine has the minimum maximum processing time
    schedule = []
    if processing_times[min_max_job][0] < processing_times[min_max_job][1]:
        machine_order = [0, 1]
    else:
        machine_order = [1, 0]

    # Step 4: schedule remaining jobs based on their minimum processing time on the first machine and maximum processing time on the second machine
    remaining_jobs = set(range(n)) - set([min_max_job])
    jobs_to_schedule = sorted(remaining_jobs, key=lambda x: (processing_times[x][machine_order[0]], processing_times[x][machine_order[1]], x))
    schedule.append(min_max_job)
    schedule.extend(jobs_to_schedule)
    return schedule


In [84]:
jobs =  [
    [3, 1, 4, 12],  #job0 processing time on each machines in machine order 1,2,3,4
    [8, 0, 5, 15],  #job1
    [11, 3, 8, 10], 
    [4, 7, 3, 8],
    [5, 5, 1, 10],
    [10, 2, 0, 13],
    [2, 5, 6, 9]
]

print( "the optimal sequence from Jhonsons method is:",johnsons_algorithm(7,4,jobs))
print("Makespan of this sequence is:",makespan(johnsons_algorithm(7,4,jobs),jobs))
print("Makespan for this sequence by Or-Tools is:", 85)

the optimal sequence from Jhonsons method is: [3, 6, 0, 4, 1, 5, 2]
Makespan of this sequence is: 91
Makespan for this sequence by Or-Tools is: 85


In [15]:
# job_tuples = []
# for job in jobs:
#     job_list = []
#     for i, time in enumerate(job):
#         job_list.append((i, time))
#     job_tuples.append(job_list)


In [36]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/drive
